In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score, precision_score, recall_score
import time
#HP Tuning
from tensorboard.plugins.hparams import api as hp

%load_ext tensorboard

In [2]:
InputData = np.loadtxt("dataInputLarge.csv",delimiter=",")
OutputData = np.loadtxt ("dataOutputLarge.csv", delimiter=",")

InputDataSmall = np.loadtxt("dataInput2.csv",delimiter=",")
OutputDataSmall = np.loadtxt ("dataOutput2.csv", delimiter=",")

InputDataLarge = np.loadtxt("dataInputLarge2.csv",delimiter=",")
OutputDataLarge = np.loadtxt ("dataOutputLarge2.csv", delimiter=",")

InputDataLarge.shape

(22012, 31)

In [3]:
InputDataTrain = InputData[0:9000]  
InputDataTest = InputData[9000:11006]  
OutputDataTrain = OutputData[0:9000] 
OutputDataTest = OutputData[9000:11006]

InputDataTrainS = InputDataSmall[0:900]  
InputDataTestS = InputDataSmall[900:1106]  
OutputDataTrainS = OutputDataSmall[0:900] 
OutputDataTestS = OutputDataSmall[900:1106]

InputDataTrainL = InputDataLarge[0:18000]  
InputDataTestL = InputDataLarge[18000:22012]  
OutputDataTrainL = OutputDataLarge[0:18000] 
OutputDataTestL = OutputDataLarge[18000:22012]

In [35]:
scaler = MinMaxScaler(feature_range=(0, 1))

InputDataTrain_transformed = scaler.fit_transform(InputDataTrain) 
InputDataTrain_retransformed = scaler.inverse_transform(InputDataTrain_transformed)
InputDataTest_transformed = scaler.fit_transform(InputDataTest)
InputDataTest_retransformed = scaler.inverse_transform(InputDataTest_transformed)
OutputDataTrain_transformed = scaler.fit_transform(OutputDataTrain)
OutputDataTrain_retransformed = scaler.inverse_transform(OutputDataTrain_transformed)
OutputDataTest_transformed = scaler.fit_transform(OutputDataTest)
OutputDataTest_retransformed = scaler.inverse_transform(OutputDataTest_transformed)         


InputDataTrainS_transformed = scaler.fit_transform(InputDataTrainS) 
InputDataTestS_transformed = scaler.fit_transform(InputDataTestS)
OutputDataTrainS_transformed = scaler.fit_transform(OutputDataTrainS)
OutputDataTestS_transformed = scaler.fit_transform(OutputDataTestS)

InputDataTrainL_transformed = scaler.fit_transform(InputDataTrainL) 
InputDataTestL_transformed = scaler.fit_transform(InputDataTestL)
InputDataTestL_retransformed = scaler.inverse_transform(InputDataTestL_transformed)
OutputDataTrainL_transformed = scaler.fit_transform(OutputDataTrainL)
OutputDataTestL_transformed = scaler.fit_transform(OutputDataTestL)


In [5]:
def create_model() :
    return tf.keras.models.Sequential([
      tf.keras.layers.Dense(6200,input_shape = (31,),activation='relu'),
      #tf.keras.layers.Dense(310,activation='linear'),  
      #tf.keras.layers.Dense(62,activation='linear'),
      tf.keras.layers.Dense(31),
    ])

In [6]:
def create_model_alter() :
    return tf.keras.models.Sequential([
          tf.keras.layers.Dense(1500,input_shape = (31,),activation='relu'),
          tf.keras.layers.Dense(62,activation='linear'),
          tf.keras.layers.Dense(31,activation='linear'),
    ])

In [7]:
#model_load = tf.keras.models.load_model("Models/Model")
model_load = tf.keras.models.load_model("Models/ModelLarge")

In [8]:
#model = create_model()
#model= create_model_alter()
model = model_load

In [9]:
model.summary()
model.output_shape
#model.layers[1].get_config()

Model: "sequential_171"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_536 (Dense)            (None, 6200)              198400    
_________________________________________________________________
dense_537 (Dense)            (None, 31)                192231    
Total params: 390,631
Trainable params: 390,631
Non-trainable params: 0
_________________________________________________________________


(None, 31)

In [34]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=InputDataTrain_transformed, 
          y=OutputDataTrain_transformed, 
          epochs=1000,
          batch_size = 300,
          validation_data=(InputDataTest_transformed, OutputDataTest_transformed), 
          verbose = 0,
          #callbacks=[tensorboard_callback]
         )

In [35]:
results = model.evaluate(InputDataTest_transformed,OutputDataTest_transformed)
print("Test loss, Test acc:", results)


63/63 [==============================] - 0s 1ms/step - loss: 3.1741e-05 - accuracy: 0.9148
Test loss, Test acc: [3.174080848111771e-05, 0.9147557616233826]


In [36]:

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=InputDataTrainS_transformed, 
          y=OutputDataTrainS_transformed, 
          epochs=1000,
          batch_size = 100,
          validation_data=(InputDataTestS_transformed, OutputDataTestS_transformed), 
          verbose = 0,
          #callbacks=[tensorboard_callback]
         )

In [37]:
results = model.evaluate(InputDataTest_transformed,OutputDataTest_transformed)
print("Test loss, Test acc:", results)


63/63 [==============================] - 0s 1ms/step - loss: 8.6866e-05 - accuracy: 0.8729
Test loss, Test acc: [8.686613000463694e-05, 0.8728813529014587]


In [38]:
model.fit(x=InputDataTrainL_transformed, 
          y=OutputDataTrainL_transformed, 
          epochs=1000,
          batch_size = 1000,
          validation_data=(InputDataTestL_transformed, OutputDataTestL_transformed), 
          verbose = 0,
          #callbacks=[tensorboard_callback]
         )

In [39]:
results = model.evaluate(InputDataTest_transformed,OutputDataTest_transformed)
print("Test loss, Test acc:", results)



63/63 [==============================] - 0s 1ms/step - loss: 2.7093e-05 - accuracy: 0.9282
Test loss, Test acc: [2.7093021344626322e-05, 0.9282153248786926]


In [ ]:
# %tensorboard --logdir logs --host localhost --port 6003

In [65]:
#InputDataTestL[4006:4007]

In [66]:
first = 1800
second = 1801
start = time.time()
#predictions = model.predict(InputDataTest_transformed[first:second])
predictions = model.predict(InputDataTestL_transformed[4007:4008])
end = time.time()
print(end - start)
print("predictions shape:", predictions.shape)
print("InputDataTest:",InputDataTest_transformed[first:second])
print("InputDataTest in db (Command Gains)", InputDataTest_retransformed[first:second])
print("OutputDataTest:",OutputDataTest_transformed[first:second])
print("Predictions:",predictions)

print("OutputDataTest in dB:",scaler.inverse_transform(OutputDataTest_transformed[first:second]))
print("predictions in dB",scaler.inverse_transform(predictions))
print("OutputDataTest in dB:",scaler.inverse_transform(OutputDataTestL_transformed[4007:4008]))
print("predictions in dB",scaler.inverse_transform(predictions))

print("Diff:", scaler.inverse_transform(OutputDataTest_transformed[first:second])-scaler.inverse_transform(predictions))

0.041986942291259766
predictions shape: (1, 31)
InputDataTest: [[1. 1. 0. 0. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0.
  0. 0. 1. 1. 1. 0. 1.]]
InputDataTest in db (Command Gains) [[ 12.  12. -12. -12.  12. -12. -12.  12.  12.  12.  12.  12. -12. -12.
  -12. -12. -12. -12.  12. -12. -12.  12. -12. -12. -12. -12.  12.  12.
   12. -12.  12.]]
OutputDataTest: [[0.71927412 0.78904234 0.18535851 0.21602521 0.99362233 0.21163324
  0.18057789 0.78582691 0.6042948  0.60405591 0.60771794 0.79499388
  0.20244358 0.38880652 0.39039324 0.38950383 0.37881247 0.21590648
  0.99441074 0.2005424  0.20221791 0.98903036 0.2180508  0.37573703
  0.38219919 0.20263038 0.78937167 0.62962942 0.79625298 0.01763513
  0.99959126]]
Predictions: [[0.28759596 0.40088192 0.3967362  0.40270257 0.40088028 0.40567118
  0.40326497 0.4018167  0.40026242 0.40208307 0.40340424 0.40363395
  0.4019183  0.40391526 0.40453428 0.4047367  0.40207678 0.4000545
  0.40041155 0.40072602 0.40612817 0.40310067 0.4072

In [47]:
model.output_shape

(None, 31)

In [9]:
%run Code.ipynb

C:/Users/Robert/AppData/Local/Temp/xpython_12460/2936004681.py:57: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  Gopt_db = np.linalg.lstsq(leak.conj().T, G_db2)[0]
C:/Users/Robert/AppData/Local/Temp/xpython_12460/4126492185.py:34: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  num = np.array([(1+G*beta), -2*cos(w0), (1-G*beta)]/(1+beta))
C:/Users/Robert/AppData/Local/Temp/xpython_12460/4126492185.py:35: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays wi

In [63]:
commandGains = InputDataTest_retransformed[first:second].reshape((31,1)) 
commandGains = InputDataTestL_retransformed[4007:4008].reshape((31,1)) 
#print(commandGains)
filterGainsPredicted = scaler.inverse_transform(predictions).reshape((31,1)) 
#print(filterGainsPredicted)

In [64]:
thirdOctaveGEQwithPredictions(commandGains,filterGainsPredicted)

C:/Users/Robert/AppData/Local/Temp/xpython_12460/2936004681.py:57: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  Gopt_db = np.linalg.lstsq(leak.conj().T, G_db2)[0]
C:/Users/Robert/AppData/Local/Temp/xpython_12460/4126492185.py:34: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  num = np.array([(1+G*beta), -2*cos(w0), (1-G*beta)]/(1+beta))
C:/Users/Robert/AppData/Local/Temp/xpython_12460/4126492185.py:35: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays wi

In [14]:
#model.save("Models/Model")

In [45]:
model = tf.keras.models.load_model("Models/Model")

In [14]:
predictions_load = model_load.predict(InputDataTest_transformed[first:second])

NameError: name 'first' is not defined

In [30]:
start = time.time()*1000
for i in range(1):
    predictions_load = model_load.predict(InputDataTest_transformed[first:second])
    
end = time.time()*1000
print(end - start)

50.9814453125


In [72]:
start = time.time()*1000
for i in range(100):
    predictions_load = model.predict(InputDataTest_transformed[first:second])
    
end = time.time()*1000
print(end - start)

3628.62109375


In [73]:
predictions_load

array([[0.95552653, 0.01717946, 0.9753712 , 0.23048931, 0.36160302,
        0.21802326, 0.9815725 , 0.20328674, 0.21433987, 0.97285426,
        0.01037176, 0.77131665, 0.62116945, 0.7872682 , 0.2096115 ,
        0.2113091 , 0.79348993, 0.62303555, 0.78559655, 0.00301883,
        0.780452  , 0.62151444, 0.61618125, 0.79255044, 0.20644717,
        0.2096918 , 0.7820731 , 0.6277895 , 0.79709864, 0.01824743,
        0.9954647 ]], dtype=float32)

In [80]:
start = time.time()*1000
for i in range(100):
    prediction_new = model(InputDataTest_transformed[first:second])
    pnew_retrans = scaler.inverse_transform(prediction_new).reshape((31,1)) 
    end = time.time()*1000
print(end - start)

79.97119140625


In [6]:
pnew_retrans

NameError: name 'pnew_retrans' is not defined

In [22]:
a1 = np.array([12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12]).reshape(31,1)    
a2 = np.array([-12,-12,-12,-12,-12,-12,-12,-12,-12,-12,-12,-12,-12,-12,-12,-12,-12,-12,-12,-12,-12,-12,-12,-12,-12,-12,-12,-12,-12,-12,-12]).reshape(31,1)    
a3 = np.array([12,-12,12,-12,12,-12,12,-12,12,-12,12,-12,12,-12,12,-12,12,-12,12,-12,12,-12,12,-12,12,-12,12,-12,12,-12,12]).reshape(31,1)
a4 = np.array([-12,12,-12,12,-12,12,-12,12,-12,12,-12,12,-12,12,-12,12,-12,12,-12,12,-12,12,-12,12,-12,12,-12,12,-12,12,-12]).reshape(31,1)  
a5 = np.array([12,-12,-12,-12,12,12,12,-12,-12,-12,12,12,12,-12,-12,-12,12,12,12,-12,-12,-12,12,12,12,-12,-12,-12,12,12,12]).reshape(31,1)  
a6 = np.array([-12,12,12,12,-12,-12,-12,12,12,12,-12,-12,-12,12,12,12,-12,-12,-12,12,12,12,-12,-12,-12,12,12,12,-12,-12,-12]).reshape(31,1)  

a = np.array([[a1],[a2],[a3],[a4],[a5],[a6]]).reshape(6,31)

In [23]:
at = scaler.fit_transform(a)

In [33]:
commandGains = a3
ap = model.predict(at[2:3])
filterGainsPredicted = scaler.inverse_transform(ap).reshape((31,1)) 

In [34]:
thirdOctaveGEQwithPredictions(commandGains,filterGainsPredicted)

C:/Users/Robert/AppData/Local/Temp/xpython_11480/2936004681.py:57: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  Gopt_db = np.linalg.lstsq(leak.conj().T, G_db2)[0]
C:/Users/Robert/AppData/Local/Temp/xpython_11480/4126492185.py:34: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  num = np.array([(1+G*beta), -2*cos(w0), (1-G*beta)]/(1+beta))
C:/Users/Robert/AppData/Local/Temp/xpython_11480/4126492185.py:35: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays wi